In [221]:
testEntities_crf='consolidated2/consolidated/dev+test+context.new.crf.p'
trainEntities_crf='consolidated2/consolidated/train+context.new.crf.p'

testEntities_ent='consolidated/dev+test+context.5.p'
trainEntities_ent='consolidated/train+context.5.p'

train_articles_crf, train_titles_crf, train_identifiers_crf, train_downloaded_articles_crf, \
TRAIN_ENTITIES_CRF, TRAIN_CONFIDENCES_CRF, TRAIN_COSINE_SIM_CRF, CONTEXT1_crf, CONTEXT2_crf = pickle.load(open(trainEntities_crf, "rb"))

#load cached entities (speed up)
train_articles, train_titles, train_identifiers, train_downloaded_articles, \
TRAIN_ENTITIES, TRAIN_CONFIDENCES, TRAIN_COSINE_SIM, CONTEXT1, CONTEXT2 = pickle.load(open(trainEntities_ent, "rb"))


test_articles_crf, test_titles_crf, test_identifiers_crf, test_downloaded_articles_crf,\
TEST_ENTITIES_CRF, TEST_CONFIDENCES_CRF, TEST_COSINE_SIM_CRF, CONTEXT_crf, CONTEXT2_crf = pickle.load(open(testEntities_crf, "rb"))

test_articles, test_titles, test_identifiers, test_downloaded_articles,\
TEST_ENTITIES, TEST_CONFIDENCES, TEST_COSINE_SIM, CONTEXT, CONTEXT2 = pickle.load(open(testEntities_ent, "rb"))



In [350]:
from sklearn.linear_model import LogisticRegression as MaxEnt
import copy
import random
import collections
from itertools import izip
import sys, json, pdb, pickle, operator, collections
import predict2 as p
import warnings
import random

warnings.filterwarnings("ignore")



def dd():
    return {}

def ddd():
    return collections.defaultdict(dd)

class Classifier(object):

    def __init__(self, TRAIN_ENTITIES, TRAIN_CONFIDENCES, TRAIN_COSINE_SIM,\
                 TEST_ENTITIES, TEST_CONFIDENCES, TEST_COSINE_SIM):
        self.TRAIN_ENTITIES = TRAIN_ENTITIES
        self.TRAIN_CONFIDENCES = TRAIN_CONFIDENCES
        self.TRAIN_COSINE_SIM = TRAIN_COSINE_SIM

        self.TEST_ENTITIES = TEST_ENTITIES
        self.TEST_CONFIDENCES = TEST_CONFIDENCES
        self.TEST_COSINE_SIM = TEST_COSINE_SIM

        self.match_orig_feature = True
        self.print_query_scores = False


    def trainClassifier(self, train_identifiers):

        classifier = MaxEnt()
        X = []
        Y = []
        for article_index in range(len(self.TRAIN_ENTITIES)):
            article = self.TRAIN_ENTITIES[article_index]
            for query_index in range(len(article)):
                query = article[query_index]
                for supporting_article_index in range(1,len(query)):
                    features = self.getFeatures(article_index, query_index, supporting_article_index, \
                                                self.TRAIN_ENTITIES, self.TRAIN_CONFIDENCES,self.TRAIN_COSINE_SIM, CONTEXT1)
                    labels = self.getLabels(article_index, query_index, supporting_article_index, \
                                                self.TRAIN_ENTITIES, train_identifiers)
                        
                    for label in labels:                            
                        X.append(features)
                        Y.append(label)
                assert( len(X) == len(Y))
            

        print "Num training examples", len(X)
        print "Dist of classes", [sum([i == y for y in Y])*1./len(X) for i in range(6)]
        classifier.fit(X,Y)
            
        return classifier

    def predictEntities(self, classifier):
        predictions = [0,0,0,0,0,0]
        DECISIONS = copy.deepcopy(self.TEST_ENTITIES)
        i = 0
        for article_index in range(len(self.TEST_ENTITIES)):
            article = self.TEST_ENTITIES[article_index]
            for query_index in range(len(article)):
                query = article[query_index]
                for supporting_article_index in range(len(query)):
                    if supporting_article_index == 0:
                        DECISIONS[article_index][query_index]\
                            [supporting_article_index] = [1, 1, 1, 1]
                        continue
                    DECISIONS[article_index][query_index]\
                            [supporting_article_index] = [0, 0, 0, 0] 

                    features = self.getFeatures(article_index, query_index, supporting_article_index, self.TEST_ENTITIES, self.TEST_CONFIDENCES,\
                               self.TEST_COSINE_SIM, CONTEXT2)

#                     assert len(features) == 41
                    prediction = classifier.predict(features)[0]
                    predictions[prediction] += 1
                    if prediction < 4:
                        DECISIONS[article_index][query_index]\
                            [supporting_article_index][prediction] = 1
                    elif prediction == 4:
                        DECISIONS[article_index][query_index]\
                            [supporting_article_index] = [1, 1, 1, 1]
                    if i < 10:
                        print DECISIONS[article_index][query_index]\
                            [supporting_article_index]
                        i+= 1
        print [p*1. / sum(predictions) for p in predictions]

        return DECISIONS

    #Run both Max Confidence and Majority Aggregation Schemes given the decisions
    #Return the decided tag for each query
    def aggregateResults(self, DECISIONS):
        majority = []
        max_conf = []
        for article_index in range(len(self.TEST_ENTITIES)):
            max_conf.append([])
            majority.append([])
            article = self.TEST_ENTITIES[article_index]
            for query_index in range(len(article)):
                max_conf[article_index].append([])
                majority[article_index].append([])
                query = article[query_index]
                for entity_index in range(4):
                    max_confidence = -1
                    max_confidence_tag = ''
                    tag_occurances = {}
                    for supporting_article_index in range(len(query)):
                        supporting_article = query[supporting_article_index]
                        if DECISIONS[article_index][query_index][supporting_article_index]\
                           [entity_index] == 0:
                            continue


                        confidence = self.TEST_CONFIDENCES[article_index][query_index]\
                                [supporting_article_index][entity_index]
                        entity = supporting_article[entity_index].strip().lower()
#                         assert(not entity == '')

                        ##Update counts of majority
                        if entity not in tag_occurances:
                            tag_occurances[entity] = 1
                        else:
                            tag_occurances[entity] += 1

                        ##Update max_confidence
                        if confidence > max_confidence:
                            max_confidence = confidence
                            max_confidence_tag = entity
                    max_majority_count = -1
                    majority_tag = ''
                    for ent in tag_occurances:
                        if tag_occurances[ent] > max_majority_count:
                            max_majority_count = tag_occurances[ent]
                            majority_tag = ent
                    max_conf[article_index][query_index].append(max_confidence_tag)
                    majority[article_index][query_index].append(majority_tag)

        return majority, max_conf



    def evaluateBaseline(self, predicted_identifiers, test_identifiers, COUNT_ZERO):
        for entity_index in range(4):
            num_queries = 5
            predicted_correct = [0.] * num_queries
            total_predicted   = [0.] * num_queries
            total_gold        = [0.] * num_queries

            for article_index in range(len(predicted_identifiers)):
                ## TODO: Add classifier for selecting query index?
                for query_index in range(len(predicted_identifiers[article_index])):        
                    predicted = predicted_identifiers[article_index][query_index][entity_index].strip().lower()
                    gold = test_identifiers[article_index][entity_index].strip().lower()
                    if gold == '' or (not COUNT_ZERO and gold == 'zero'):
                        continue


                    #special handling for shooterName (lenient eval)
                    if entity_index == 0:
                        predicted = set(predicted.split('|'))
                        gold = set(gold.split('|'))
                        correct = gold.intersection(predicted)
                        predicted_correct[query_index] += (1 if len(correct)>0 else 0)
                        total_predicted[query_index] += 1
                        total_gold[query_index] += 1 
                    else:
                        total_predicted[query_index] += 1
                        if predicted == gold:
                            predicted_correct[query_index] += 1
                        total_gold[query_index] += 1


            print "Entity", entity_index, ":",
            if sum(total_predicted) == 0 :
                continue

            if sum(predicted_correct) == 0 :
                continue

            if  self.print_query_scores:
                print "BEGINNING WITH PER QUERY SCORES"

                for query_index in range(num_queries):
                    print "*********************************************"
                    print
                    print "QUERY INDEX:", query_index
                    self.displayScore(predicted_correct[query_index], total_predicted[query_index],\
                                      total_gold[query_index])
                    print
                    print "*********************************************"
                print "NOW SHOWING SCORES AGGREGATED OVER ALL QUERRIES"
            self.displayScore(sum(predicted_correct), sum(total_predicted),sum(total_gold))

    def displayScore(self, predicted_correct, total_predicted, total_gold):
        precision = predicted_correct / total_predicted
        recall = predicted_correct / total_gold
        f1 = (2*precision*recall)/(precision+recall)
        print "PRECISION", precision, "RECALL", recall, "F1", f1
        print "Total predicted", total_predicted

    def runExploratoryTests(self, DECISIONS, train_identifiers, test_identifiers):
        print "Exploring how many times gold entity is not in original document"
        count = collections.defaultdict(lambda:0.)
        total_count = collections.defaultdict(lambda:0.)
        for article_index in range(len(self.TRAIN_ENTITIES)):
            article = self.TRAIN_ENTITIES[article_index]
            for entity_index in range(4):
                for query_index in range(len(article)):
                    query = article[query_index]
                    if query_index > 0: #not shooter
                        orig_entity = query[0][entity_index].strip().lower()
                        gold = train_identifiers[article_index][entity_index].strip().lower()
                        for supp_index in range(len(query)):
                            entity = query[supp_index][entity_index].strip().lower()
                            if entity == gold:
                                count[entity_index] += 1
                            total_count[entity_index] +=1
                    else:
                        orig_entity = set(query[0][entity_index].strip().lower().split('|'))
                        gold = set(train_identifiers[article_index][entity_index].strip().lower().split('|'))
                        for supp_index in range(len(query)):
                            entity = set(query[supp_index][entity_index].strip().lower().split('|'))
                            if len(entity.intersection(gold)) > len(orig_entity.intersection(gold)):
                                count[entity_index] += 1
                            total_count[entity_index] +=1

        print "COUNT ", count
        print "TOTAL ", total_count
        print "Ratio" , [a/b for a,b in izip(count.values(),total_count.values())]

        print "Exploring if classifier ever chooses not first entity"
        print "Program will halt with assert if classifier chooses entity not in org document"
        ones = [0] * 4
        ones_not_orig = [0] * 4
        counts = [0] * 4
        for entity_index in range(4):
            for article_index in range(len(self.TEST_ENTITIES)):
                article = self.TEST_ENTITIES[article_index]
                for query_index in range(len(article)):
                    query = article[query_index]
                    orig_entity = query[0][entity_index].strip().lower()
                    gold = test_identifiers[article_index][entity_index].strip().lower()
                    for supp_index in range(len(query)):
                        decision = DECISIONS[article_index][query_index][supp_index][entity_index]
                        ones[entity_index] += decision
                        counts[entity_index] += 1
                        if decision == 1:
                            entity = query[supp_index][entity_index].strip().lower()
                            if entity != orig_entity:
                                ones_not_orig[entity_index] += 1

        print "Ratio ones in prediction", [ ones[x]*1. / counts[x] for x in range(4)]
        print "Ratio one not matching original entity in prediction", [ ones_not_orig[x]*1. / counts[x] for x in range(4)]
        print "It does not"


    def trainAndEval(self, train_identifiers, test_identifiers, COUNT_ZERO):
        classifier = self.trainClassifier(train_identifiers)
        DECISIONS  = self.predictEntities(classifier)

        debug = False
        if debug:
            self.runExploratoryTests(DECISIONS, train_identifiers, test_identifiers)
            return

        majority, max_conf = self.aggregateResults(DECISIONS)
        print "#############################################################"
        print "Evaluation for Classifier baseline with MAJORITY aggregation"
        print
        self.evaluateBaseline(majority, test_identifiers, COUNT_ZERO)

        print
        print "#############################################################"
        print "Evaluation for Classifier baseline with MAX CONFIDENCE aggregation"
        print
        self.evaluateBaseline(max_conf, test_identifiers, COUNT_ZERO)
        print
        print "#############################################################"
    
    def getFeatures(self, article_index, query_index, supporting_article_index, entities, confidences, cosine_sim, context):        
        features= []

        #Construct feature vector for this sampled entity
        original_confidence = confidences[article_index][query_index][0]
        confidence = confidences[article_index][query_index][supporting_article_index]
        
        #One hot vector to show if entity matches orginal
        original_entity = entities[article_index][query_index][0]
        new_entity = entities[article_index][query_index][supporting_article_index]
        match_features = []
        for e_index in range(len(original_entity)):
            if original_entity[e_index] == '':
                match_features += [0, 1]
            elif original_entity[e_index].strip().lower() == new_entity[e_index].strip().lower():
                match_features += [1, 0]
            else:
                match_features += [0, 1]
        
        # Cosine sim array is shifted by one.
        # Index 0 should be 1 as orig is same as itself.
        tfidf = 1 if supporting_article_index == 0 else \
                cosine_sim[article_index]\
                [query_index][supporting_article_index - 1]

#         features = original_confidence+ confidence + match_features + [tfidf]
        features = original_confidence+ confidence  + [tfidf]

        for c in context[article_index][query_index][supporting_article_index]:
            features += c            
#         assert len(features) == 41
        
        return features
    
    def getLabels(self, article_index, query_index, supporting_article_index, entities, identifier):
        #Extract out label for this article (ie. is label correct)
        labels = []
        gold_entities = identifier[article_index]
        new_entities      = entities[article_index][query_index][supporting_article_index]
        
        for ind in range(len(gold_entities)):
            ent = new_entities[ind].lower().strip()
            gold = gold_entities[ind].lower().strip()
            if gold == "":
                continue
            if ent == "":
                continue
            
            #special handling for shooterName (entity_index = 0)
            if ind == 0:
                new_person = set(ent.split('|'))
                gold_person = set(gold.split('|'))
                if len(new_person.intersection(gold_person)) > 0:
                    labels.append(ind)
            else:
                if gold == ent:
                    labels.append(ind)
        if labels == [0, 1, 2, 3]:
            labels = [4]
        elif labels == []:
            labels = [5]
        
        assert (len(labels) > 0)
        return labels
    
    

In [352]:
i = 12
j = 0
k = 2
for i in [random.randint(0,len(TEST_ENTITIES_CRF))]:
    print "TEST ENTITIES",     TEST_ENTITIES[i][j][k]
    print "TEST ENTITIES CRF", TEST_ENTITIES_CRF[i][j][k]
    print "GOLD"            , test_identifiers [i]
    print "$$$$$$$$$$$$$$"


verbose = False

count_name = 0
print "size", len(TEST_CONFIDENCES_CRF)

for entity_ind in range(4):
    entity_name = p.int2tags[entity_ind+1]
    correct = 0
    gold_num = 0
    total = 0
    for article_ind in range(len(TEST_ENTITIES_CRF)):
        article = TEST_ENTITIES_CRF[article_ind]
        gold_ent = test_identifiers[article_ind][entity_ind].strip().lower()
        for query_ind in range(len(article)):
            query = article[query_ind]
#             for sup_ind in range(len(query)):
                

            sup = query[0]

            ent = sup[entity_ind].strip().lower()
            
            if gold_ent == '' or (True and gold_ent == 'zero'):
                    continue
            if entity_ind == 0:
                    gold_ent_set = set(gold_ent.split('|'))
                    correct_int = gold_ent_set.intersection(ent)
                    correct += (1 if len(correct_int)>0 else 0)
                    if not ent == set(['']):
                        count_name += 1
#                         print "ent", ent
#                         print "gold", gold_ent_set
                    gold_num += 1
                    total += 1
            else:

                if ent == gold_ent: 
                    correct += 1
#                 else:
#                     if not ent == 'zero':
#                         print "---------------"
#                         print "GOLD", gold_ent
#                         print "ENT", ent
                gold_num += 1
                total   += 1
#     if entity_ind == 1:
#         continue
    prec = correct*1./total
    recall = correct*1./gold_num
#     print entity_name
#     print "prec", prec
#     print 'recall', recall
    f1 = 0
    if not prec + recall == 0:
        f1 = 2*(prec*recall)/(prec+recall)
#     print
#     print entity_name
#     print "prec", prec, "recall", recall, "f1:" , f1
            
# baseline = Classifier(TRAIN_ENTITIES_CRF, TRAIN_CONFIDENCES_CRF, TRAIN_COSINE_SIM_CRF,\
#              TEST_ENTITIES_CRF, TEST_CONFIDENCES_CRF, TEST_COSINE_SIM_CRF)
   
baseline = Classifier(TRAIN_ENTITIES, TRAIN_CONFIDENCES, TRAIN_COSINE_SIM, \
                      TEST_ENTITIES, TEST_CONFIDENCES, TEST_COSINE_SIM)
            

baseline.trainAndEval(train_identifiers, test_identifiers, False)


TEST ENTITIES ['', 'one', 'four', 'phoenix']
TEST ENTITIES CRF ['', 'one', 'four', '']
GOLD ['', 'zero', 'five', 'Atlanta']
$$$$$$$$$$$$$$
size 292
Num training examples 16077
Dist of classes [0.04447347141879704, 0.35784039310816695, 0.19437706039684022, 0.1659513590844063, 0.012004727250108852, 0.22535298874168067]
[0, 0, 1, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0.004175101214574899, 0.43294534412955465, 0.07300101214574899, 0.038461538461538464, 0.00037955465587044535, 0.45103744939271256]
#############################################################
Evaluation for Classifier baseline with MAJORITY aggregation

Entity 0 : PRECISION 0.452380952381 RECALL 0.452380952381 F1 0.452380952381
Total predicted 210.0
Entity 1 : PRECISION 0.687654320988 RECALL 0.687654320988 F1 0.687654320988
Total predicted 810.0
Entity 2 : PRECISION 0.684870848708 RECALL 0.684870848708 F1 0.684870848708
Total predicted 1355.0


In [240]:
print len(CONTEXT1[0][0][0][1])


Entity 0 : PRECISION 0.142857142857 RECALL 0.142857142857 F1 0.142857142857
Total predicted 210.0
Entity 1 : PRECISION 0.501234567901 RECALL 0.501234567901 F1 0.501234567901
Total predicted 810.0
Entity 2 : PRECISION 0.287084870849 RECALL 0.287084870849 F1 0.287084870849
Total predicted 1355.0
Entity 3 : PRECISION 0.265068493151 RECALL 0.265068493151 F1 0.265068493151
Total predicted 1460.0


6
